In [21]:
import requests
import datetime
from datetime import datetime, time, timedelta
import numpy as np

date_time_format = "%Y-%m-%dT%H:%M:%S.%fZ"
expiries_url = "https://live.markethound.in/api/history/expiries?index=NIFTY"
expiry_dates = []

# Get expiry dates
response = requests.get(expiries_url)
data = response.json()
for date_str in data['result']:
    expiry_dates.append(datetime.strptime(date_str, date_time_format).date())

# Filter expiry dates for which we have data
today = datetime.strptime("2024-03-11T00:00:00.000Z", date_time_format).date()
expiry_dates = [date for date in expiry_dates if date >= today]

input_data = {}
output_data = {}

# Function to fetch data for given expiry date and days to expiry
def fetch_data(date, dte):
    start_time = datetime.combine(date, time(hour=14, minute=0))
    end_time = datetime.combine(date, time(hour=15, minute=0))
    current_time = start_time
    time_increment = timedelta(minutes=15)
    data = []

    while current_time <= end_time:
        datetime_str = current_time.strftime(date_time_format)
        url = f"https://live.markethound.in/api/history/decay?name=NIFTY&expiry={datetime_str}&dte={dte}"
        response = requests.get(url)
        if response.status_code != 200:
            break
        result = response.json().get('result', [])
        if len(result) == 0:
            break
        price_data = [result[0].get(key, 0) for key in ['open', 'close', 'high', 'low']]
        data.append(price_data)
        current_time += time_increment

    if len(data) != 0:
        return np.array(data)
    else:
        return None

# Fetch data for each expiry date
for date in expiry_dates:
    input_data[date] = fetch_data(date, 3)
    output_data[date] = fetch_data(date, 2)


In [22]:
input_data

{datetime.date(2024, 3, 14): array([['260.85', '230.55', '260.85', '230.55'],
        ['260.85', '230.55', '260.85', '230.55'],
        ['260.85', '230.55', '260.85', '230.55'],
        ['260.85', '230.55', '260.85', '230.55'],
        ['260.85', '230.55', '260.85', '230.55']], dtype='<U6')}

In [23]:
output_data

{datetime.date(2024, 3, 14): array([['220.70', '198.55', '233.15', '195.25'],
        ['220.70', '198.55', '233.15', '195.25'],
        ['220.70', '198.55', '233.15', '195.25'],
        ['220.70', '198.55', '233.15', '195.25'],
        ['220.70', '198.55', '233.15', '195.25']], dtype='<U6')}

The given data has only one data point for each variable and hence it is not feasible to formulate a ML algorithm to learn the pattern.

And hence no algorithm is stated